In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import csv
import cv2
import imageio.v2 as iio
import statistics
from math import sqrt, cos, sin, pi

import sys

racine = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(racine)

from config import chemin_tt_espace

from PIL import Image, ImageDraw, ImageFont
from moviepy.editor import VideoFileClip, CompositeVideoClip, clips_array, ColorClip

from Scripts.generation_grille_table import quadrillage_sur_image
from Scripts.fonctions_positions import coordonnees, ajout_vecteurs_trajectoires, atteintes_lateralite, nuage_frappes, banane_alexis, banane_alexis_cd_r, angles_avant_bras, bdl
from Scripts.fonctions_inertie import heatmap
from Scripts.fonctions_time import time_to_point, frappe
from Scripts.creation_gifs import gif_evol_surf_jA, gif_evol_vit_jA, gif_evol_surf_vit_jA, gif_evol_angle_jA, gif_point

In [ ]:
match="FAN-ZHENDONG_vs_TRULS-MOREGARD"
match='ALEXIS-LEBRUN_vs_MA-LONG'
point="set_1_point_0"

# les dossiers heatmap et evolutions_temporelles ont vocation à stocker des images de façon transitoire. le but est de pour avoir accès aux images qui composent un gif/une vidéo mais on en n'a pas forcément besoin dans le temps donc lorsqu'on a fini d'étudier un point et qu'on passe au suivant, les nouvelles images s'écrivent par dessus.


os.makedirs(os.path.join(chemin_tt_espace,"example",match,point,"output","heatmap"), exist_ok='True')
os.makedirs(os.path.join(chemin_tt_espace,"example",match,point,"output","evolutions_temporelles"), exist_ok='True')
os.makedirs(os.path.join(chemin_tt_espace,"example",match,point,"output","evolutions_temporelles","angle"), exist_ok='True')
os.makedirs(os.path.join(chemin_tt_espace,"example",match,point,"output","evolutions_temporelles","combinaison"), exist_ok='True')
os.makedirs(os.path.join(chemin_tt_espace,"example",match,point,"output","evolutions_temporelles","surface_inertie"), exist_ok='True')
os.makedirs(os.path.join(chemin_tt_espace,"example",match,point,"output","evolutions_temporelles","vitesse"), exist_ok='True')

pos_init_b, vitesse_b, pos_init_jA, vitesse_jA, pos_init_jB, vitesse_jB = coordonnees(match, point, lissage='oui')

nb_pts=len(pos_init_b) # pour tracer les courbes d'évolution au cours du temps
time_points = np.array([f*1/25 for f in range(len(pos_init_b))]) # on a la valeur de la surface atteignable en moins de tlim pour chaque frame et 1 frame dure 1/25s d'où cette discrétisation du temps
xmax=(len(time_points)-1)/25


with open(os.path.join(chemin_tt_espace,"example",match,point,point+"_annotation.csv"), newline='') as fichier :
    set_point_annotation=[]
    reader=csv.reader(fichier)
    for ligne in reader :
        set_point_annotation.append(ligne)

time_frappe=[] # joueur qui frappe, frame de frappe, coordonnées de la balle fichier annotation
for i in range(1, len(set_point_annotation)):
    if set_point_annotation[i][8]=='' :
        time_frappe.append([set_point_annotation[i][0], int(set_point_annotation[i][21]), (float(set_point_annotation[i][18]), -float(set_point_annotation[i][19]))])
    elif set_point_annotation[i][8]!='' :
        frappe_service=[set_point_annotation[i][0], int(set_point_annotation[i][21]), (float(set_point_annotation[i][18]), -float(set_point_annotation[i][19]))]

with open(os.path.join(chemin_tt_espace,"example",match,match+"_game.json"),  newline='') as fichier_game:
    data=json.load(fichier_game)
    game=[[cle,val] for cle,val in data.items()]
joueurA=game[0][1]
joueurB=game[1][1]
lateralite_jA=game[2][1]
lateralite_jB=game[3][1]

h, W = 800, 400

In [5]:

point = "set_1_point_0"
match='ALEXIS-LEBRUN_vs_MA-LONG'
#gif_point(match, point, avec_heatmap='oui', avec_banane='non',num_vitesse=1)
match="FAN-ZHENDONG_vs_TRULS-MOREGARD"
gif_point(match, point, avec_heatmap='oui', avec_banane='non',num_vitesse=1)